In [1]:
import xarray as xr
import sys
from tqdm import tqdm
import numpy as np

In [2]:
from dask.distributed import Client, progress

c = Client(n_workers=16)
c


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 16
Total threads: 32,Total memory: 251.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41759,Workers: 16
Dashboard: /proxy/8787/status,Total threads: 32
Started: Just now,Total memory: 251.20 GiB
Comm: tcp://127.0.0.1:40119,Total threads: 2
Dashboard: /proxy/42559/status,Memory: 15.70 GiB
Nanny: tcp://127.0.0.1:46353,


In [ ]:
%%time
ACCESS_data = xr.open_mfdataset('/g/data/p73/archive/CMIP6/ACCESS-CM2/bj594/history/ocn/ocean_month.nc-*', decode_times=True)

In [ ]:
# ti = int(sys.argv[1]) # This is provided by the submission script
## Define the number of BSP bins to output, where number = 2**tree_depth:
tree_depth = 4
ti = 0

if tree_depth<=4:
    window = 1440
    start = window*ti
    end = (ti+1)*window
else:
    window = 9*12
    start = window*ti+1548
    end = (ti+1)*window+1548

# Temperature
T = ACCESS_data.temp.isel(st_ocean=0).isel(time=slice(start,end))
print(T.shape)
# Salinity
S = ACCESS_data.salt.isel(st_ocean=0).isel(time=slice(start,end))
## Load the basins mask which will be used to split the Basins up:
ACCESS_mask = xr.open_mfdataset('/g/data/e14/txs156/Analysis/Min_Transform_Method/Mask/mask_ACCESS.nc',decode_times=True)

## Load the BSP bins
ACCESS_BSP_data = xr.open_mfdataset('/g/data/e14/txs156/Analysis/Min_Transform_Method/BSP/BSP_processed/*%i_bins*' %(tree_depth))

partitions = ACCESS_BSP_data.Partitions.isel(Time=slice(start,end)).mean('Basin')

Basins = ACCESS_BSP_data.Basin.values

In [19]:
da_fuzz = xr.zeros_like(S).expand_dims({'tree_depth':2**tree_depth}).assign_coords({'tree_depth':np.arange(0,2**tree_depth)})
print(da_fuzz.shape)
for i in tqdm(range(T.shape[0])):
    for j in ((range(2**tree_depth))):
        da_fuzz[j,i] = xr.where((S.isel(time=i)>partitions[i,j,0])&\
                                (T.isel(time=i)>partitions[i,j,2])&\
                                (S.isel(time=i)<=partitions[i,j,1])&\
                                (T.isel(time=i)<=partitions[i,j,3]),\
                                1, 0)

(128, 108, 300, 360)


100%|██████████| 108/108 [10:10<00:00,  5.65s/it]


In [20]:
da_fuzz.to_netcdf('Masks_processed/ACCESS_mask_%i_bins_50_70.nc' %(tree_depth))